In [1]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.metrics import euclidean_distances
from sklearn.utils import check_random_state, check_array, check_symmetric
#from sklearn.externals.joblib import Parallel, delayed, effective_n_jobs
from sklearn.isotonic import IsotonicRegression

def NMDSfunc(dissimilarities, n_dimensions=2, max_iter=300, verbose=False, eps=1e-3, random_state=None):
    '''
    dissimilarities: matrix
    eps: used for convergence
    '''
    dissimilarities = check_symmetric(dissimilarities, raise_exception=True)
    n_samples = dissimilarities.shape[0]
    random_state = check_random_state(random_state)

    sim_flat = ((1 - np.tri(n_samples)) * dissimilarities).ravel()
    sim_flat_w = sim_flat[sim_flat != 0]
    #Random initial configuration
    X = random_state.rand(n_samples * n_dimensions)
    X = X.reshape((n_samples, n_dimensions))

    old_stress = None
    ir = IsotonicRegression()
    for it in range(max_iter):
        # Compute distance and monotonic regression
        dis = euclidean_distances(X)
        dis_flat = dis.ravel()
        dis_flat_w = dis_flat[sim_flat != 0]

        # Finf disparities using monotonic regression
        disparities_flat = ir.fit_transform(sim_flat_w, dis_flat_w)
        disparities = dis_flat.copy()
        disparities[sim_flat != 0] = disparities_flat
        disparities = disparities.reshape((n_samples, n_samples))
        disparities *= np.sqrt((n_samples * (n_samples - 1) / 2) / (disparities ** 2).sum())
        # Compute stress
        stress = ((dis.ravel() - disparities.ravel()) ** 2).sum() / 2

        # Update X using the Guttman transform
        dis[dis == 0] = 1e-5
        ratio = disparities / dis
        B = - ratio
        B[np.arange(len(B)), np.arange(len(B))] += ratio.sum(axis=1)
        X = 1. / n_samples * np.dot(B, X)

        dis = np.sqrt((X ** 2).sum(axis=1)).sum()
        if verbose:
            print('iteration: %d, stress %s' % (it, stress))
        if old_stress is not None:
            if(old_stress - stress / dis) < eps:
                if verbose:
                    print('convergence: breaking at iteration %d with stress %s' % (it,stress))
                break
        old_stress = stress / dis

    return X, stress, it + 1


def NMDS(dissimilarities, n_dimensions=2, n_init=4,
            max_iter=300, verbose=False, eps=1e-3, random_state=None):
    """
    """
    dissimilarities = check_array(dissimilarities)
    random_state = check_random_state(random_state)

    best_pos, best_stress = None, None
    for it in range(n_init):
        pos, stress, n_iter_ = NMDSfunc(dissimilarities, n_dimensions=n_dimensions, max_iter=max_iter, verbose=verbose,
                eps=eps, random_state=random_state)
        print(pos)
        print(stress)
        if best_stress is None or stress < best_stress:
            best_stress = stress
            best_pos = pos.copy()
            best_iter = n_iter_
            
    return best_pos, best_stress, best_iter


class MDS():
    def __init__(self, n_dimensions=2,  n_init=4,
                 max_iter=300, verbose=False, eps=1e-3,
                 random_state=None):
        self.n_dimensions = n_dimensions
        self.n_init = n_init
        self.max_iter = max_iter
        self.eps = eps
        self.verbose = verbose
        self.random_state = random_state

    def fit(self, X, y=None):
      
        #self.fit_transform(X)
        X = check_array(X)
        self.dissimilarity_matrix_ = X
        
        self.embedding, self.stress, self.n_iter = NMDS(self.dissimilarity_matrix_,
            n_dimensions=self.n_dimensions, n_init=self.n_init,
            max_iter=self.max_iter, verbose=self.verbose,
            eps=self.eps, random_state=self.random_state)
        
        return self

In [2]:
sim = np.array([[0, 5, 3, 4],
                [5, 0, 2, 2],
                [3, 2, 0, 1],
                [4, 2, 1, 0]])
mds_clf = MDS()
a = mds_clf.fit(sim)

[[ 0.33594795 -0.3948635 ]
 [-0.30449107  0.42244858]
 [ 0.21293544  0.19959538]
 [-0.26354268 -0.24363838]]
0.00434591071834
[[-0.10964806 -0.4993237 ]
 [ 0.46767538 -0.08388328]
 [-0.22311473  0.21769579]
 [-0.15849331  0.37173996]]
0.00834074000133
[[-0.02166194  0.69384621]
 [ 0.02479511 -0.40010612]
 [ 0.16920386 -0.16021857]
 [-0.19094438 -0.14276781]]
0.00269603202407
[[ 0.39273048 -0.40545338]
 [-0.09989428  0.54874753]
 [-0.33097061 -0.14757184]
 [ 0.01986823 -0.00618865]]
0.0206533201877


In [3]:
a.stress

0.0026960320240685702

In [4]:
a.embedding

array([[-0.02166194,  0.69384621],
       [ 0.02479511, -0.40010612],
       [ 0.16920386, -0.16021857],
       [-0.19094438, -0.14276781]])

In [11]:
sim2 = np.array([[0, 1, 4, 5, 3, 4, 2, 8, 8, 5, 7, 1],
                [1, 0, 3, 8, 2, 6, 2, 7, 8, 8, 7, 2],
                [4, 3, 0, 7, 2, 3, 3, 7, 8, 6, 4, 3],
                [5, 8, 7, 0, 8, 9, 3, 8, 2, 2, 9, 5],
                [3, 2, 2, 8, 0, 6, 4, 8, 9, 6, 4, 2],
                [4, 6, 3, 9, 6, 0, 1, 3, 9, 8, 5, 3],
                [2, 2, 3, 3, 4, 1, 0, 4, 3, 6, 7, 1],
                [8, 7, 7, 8, 8, 3, 4, 0, 9, 6, 9, 5],
                [8, 8, 8, 2, 9, 9, 3, 9, 0, 4, 9, 9],
                [5, 8, 6, 2, 6, 8, 6, 6, 4, 0, 4, 3],
                [7, 7, 4, 9, 4, 5, 7, 9, 9, 4, 0, 6],
                [1, 2, 3, 5, 2, 3, 1, 5, 9, 3, 6, 0]])
mds_2 = MDS(n_dimensions=4)
b = mds_2.fit(sim2)


[[-0.15446274 -0.05486261 -0.37675584 -0.08459893]
 [-0.34387089 -0.03365131  0.0767962  -0.25901008]
 [ 0.22751285 -0.41175236  0.04107472 -0.02353109]
 [ 0.2008531   0.26972238 -0.16068742  0.37367883]
 [-0.13980691 -0.35197993  0.20817172 -0.07984224]
 [ 0.08703996  0.03921138  0.06160018 -0.42764075]
 [-0.11202944  0.10371476 -0.1709501  -0.21202828]
 [-0.34515376  0.22639233  0.24406729  0.32890332]
 [ 0.18911169  0.56199598 -0.03576814  0.08145895]
 [ 0.34748398 -0.01567346 -0.16583692  0.3204571 ]
 [ 0.26846573 -0.09445061  0.39324436  0.00489694]
 [-0.28342125 -0.28472253 -0.15721821 -0.003485  ]]
0.291767904638
[[-0.1615552  -0.20984844  0.33059106  0.07281988]
 [ 0.28849843  0.01388356 -0.0815488   0.34565777]
 [-0.27760733  0.24683838 -0.01389817 -0.27141689]
 [-0.41624992 -0.00124806 -0.04695133  0.10633468]
 [ 0.31844266 -0.31804836  0.20160683  0.01579095]
 [ 0.03361772  0.45713449 -0.1314461   0.03561651]
 [-0.03300346 -0.03561568 -0.39194784  0.21526183]
 [ 0.0339223   

In [12]:
b.stress

0.29176790463819546

In [23]:
output = []
for i in range(1,15):
    model = MDS(n_dimensions=i)
    b = model.fit(sim2)
    output.append(b)


[[ 0.1613248 ]
 [ 0.22010776]
 [-0.24936697]
 [-0.47111644]
 [ 0.14392748]
 [-0.58551965]
 [-0.45909141]
 [ 0.70227337]
 [ 0.41903686]
 [-0.29351046]
 [ 0.08254286]
 [ 0.95329699]]
5.02476039927
[[ -3.68248077e-01]
 [ -5.37090193e-01]
 [ -1.02741317e-01]
 [  2.59846789e-01]
 [ -4.60097177e-01]
 [ -3.62246151e-01]
 [ -2.47004894e-01]
 [  3.96974377e-01]
 [  2.10514737e-01]
 [  5.78397307e-01]
 [  5.45239088e-04]
 [  1.28307058e+00]]
6.64251111205
[[-0.42484338]
 [-0.30479624]
 [-0.10998874]
 [-0.33063021]
 [ 0.29841955]
 [ 0.6202277 ]
 [ 0.61024166]
 [ 0.02080137]
 [-0.30577037]
 [ 0.22034506]
 [-0.93587149]
 [ 0.61654035]]
4.89360173066
[[ 0.05438532]
 [-0.59573272]
 [-0.52836737]
 [-0.45190211]
 [ 0.69903265]
 [-0.2832129 ]
 [-0.36124379]
 [ 0.07172496]
 [ 0.2460921 ]
 [-0.13268537]
 [ 0.94324736]
 [ 0.49156664]]
5.3024902137
[[-0.45560002  0.11156799]
 [-0.4233677  -0.15757529]
 [-0.34590955  0.28652818]
 [ 0.51616966 -0.21391534]
 [-0.13966013  0.20397753]
 [ 0.31384222  0.34499841]

[[-0.08116372 -0.0678298  -0.32773542  0.0037978   0.1581202  -0.27168221
  -0.01947541]
 [ 0.24243294 -0.22136697 -0.15900442 -0.24252439  0.08081439  0.06574029
  -0.09401367]
 [-0.1776122   0.3099043   0.19318413 -0.25308049 -0.08706478 -0.06912324
  -0.07726093]
 [ 0.06902479  0.03809164 -0.02318918  0.15105365  0.32492163  0.27164187
   0.15860277]
 [-0.14084632  0.01033493 -0.10096627  0.27927767 -0.25103182  0.28215485
   0.08805928]
 [ 0.12553722 -0.08228314  0.30029728  0.13401962 -0.28027342  0.03603367
   0.21412394]
 [ 0.06617278 -0.19202705  0.0807503  -0.17818759 -0.22616181 -0.30539808
  -0.05582295]
 [ 0.02254397  0.17542147 -0.09171034  0.12196903  0.15393667 -0.16833806
   0.28573532]
 [ 0.20824536  0.30183024 -0.07538518  0.08756113 -0.04868345  0.10439224
  -0.19561271]
 [-0.27996053 -0.07153009 -0.11401656  0.12532452 -0.1714214  -0.05548782
  -0.24297598]
 [-0.16615943 -0.11485241 -0.00933148 -0.20292242  0.06048165  0.25398286
   0.14342853]
 [ 0.13897796 -0.0945

[[ 0.19081803  0.04061966  0.00521713 -0.10849267  0.22396832  0.04294812
  -0.05768113  0.13660138 -0.0725113   0.29133688  0.07182149]
 [-0.01306062 -0.14767178  0.09203194 -0.02743385 -0.22814512  0.17441614
  -0.21368117 -0.15290823 -0.0982151  -0.05439156  0.19614244]
 [-0.15309847  0.18236897  0.01362402  0.05098922  0.19376007  0.06023862
  -0.19345358  0.00600035  0.05373259  0.07530522 -0.26502866]
 [ 0.17283212  0.07165452 -0.12123787 -0.16184358 -0.0901224  -0.19721869
  -0.04359696  0.27152791  0.00637164 -0.1195531   0.16236006]
 [-0.14633839 -0.08527627  0.07755405 -0.15558635 -0.07569095  0.06151591
   0.15482886  0.10825173  0.25065728  0.10006596  0.19740053]
 [-0.15237437 -0.09837149  0.28311466 -0.05563182  0.04926844 -0.06680502
  -0.08987165 -0.14893281 -0.04860157  0.1722531  -0.08167685]
 [ 0.11567274  0.15331908  0.0061233  -0.16754715  0.08259481  0.1846121
   0.08400649  0.11722048 -0.10609965 -0.19163249 -0.10994924]
 [ 0.06618273 -0.12278285 -0.15758525  0.1

In [26]:
for i in range(0,14):  
    print(output[i].stress)


4.89360173066
1.8792696259
1.21326647196
0.648958687518
0.472625524899
0.251811386442
0.232308127696
0.181478499568
0.168010556649
0.126736942674
0.139525192311
0.0929909224331
0.122592649375
0.0925085461005
